The first step in the analysis is to beamform over the points we are interested in.

Delay and sum beamforming

This approach consists in delaying the signal recieved by each microphone by the time sound would take to reach it from the point we are testing.

$t_0 = \lvert x - x_0\rvert / c_0$

The delayed sounds are then summed and the result is divided by the number of microphones in the array. In this way if the sound source corresponds to the point we are testing the delayed signals will produce constructive interference producing an amplified output, otherwise the phase mismatch will cancel the signals and produce a low amplitude output. 

The output of the beamformer is expressed as

$\begin{equation}Z(t, x_0) = \frac{4\pi}{M} \sum_{m=1}^M \lvert x - x_0 \rvert p_m(x_0, t + t_0) \end{equation}$

where $p_m$ if the pressure measured by microphone $m$, $M$ is the number of microphones, $x_0$ is the test position and $x$ is the microphone position.

Time domain is simple and computationally fast, but it doesn't allow for post processing deconvolution techniques, for this reason we will use the frequency domain formulation.

Frequency domain formulation
The first step of this approach is to compute the fourier transform of the signals recieved by the microphones.

$P_m(x, x_0, \omega) = \mathscr{F}\{p(x_0, t)\} = \frac{Q(\omega)e^{-i \omega t_0}}{4\pi \lvert x - x_0 \rvert}$

The output of the beamformer is now

$\begin{equation}Z(\omega, x_0) = \frac{4\pi}{M} \sum_{m=1}^M \lvert x - x_0 \rvert P(x, x_0, \omega) e^{i \omega t_0} \end{equation}$

where

$\begin{equation}s(x, x_0, \omega) = 4\pi \lvert x - x_0 \rvert e^{i \omega t_0} \end{equation}$

is called the steering function.

Given a point in the searching grid $x_0$ there is a steering function for each microphone, which are usually summarized in the steering vector


$\begin{equation}
  g(x, \omega) =
  \left(
    \begin{aligned}
      & 4\pi \lvert x_1 - x_0 \rvert e^{-i \omega t_1} \\
      & 4\pi \lvert x_2 - x_0 \rvert e^{-i \omega t_2} \\
	  & \;\;\;\;\;\;\;\;\;\;\;\;\;\vdots \notag \\
	  & 4\pi \lvert x_M - x_0 \rvert e^{-i \omega t_M}
    \end{aligned}
  \right)
\end{equation}$

The calculations become more convenient if the vector is normalized by the distance between the source and the center of the microphone array, so the normalilzed steering vector is defined as

$\begin{equation}
\hat{e}(x, \omega) = \frac{g(x, \omega)}{\lvert x_c - x_0 \rvert}
\end{equation}$

The fourier transform of the microphone array can also be represented in vector form

$\begin{equation}
Y(\omega) = 
\left(
    \begin{aligned}
	& P_1(\omega) \\
	& P_2(\omega) \\
	& \;\;\;\;\;\vdots \notag \\
	& P_M(\omega)
    \end{aligned}
  \right)
\end{equation}$

The output of the transformer can then be written as the dot product between the two vectors

$\begin{equation}
Z(\omega) = \frac{g(x, \omega)^\dag Y(\omega)}{M}
\end{equation}$

We are interested in the power output of the beamformer, so

$\begin{equation}
L(x) = ZZ^* = \frac{g^\dag(YY^\dag)g}{M^2}
\end{equation}$

The term in the middle of the equation is called the Cross Spectral Matrix

$\begin{equation}
CSM(\omega) = Y(\omega)Y(\omega)^\dag =
\begin{bmatrix} 
Y_1Y_1^*           & \dots         & Y_1Y_M^*         \\ 
\vdots   &  \ddots    & \vdots \\
Y_MY_1^*           & \dots         & Y_MY_M^*         \\
\end{bmatrix}
\end{equation}$


Deconvolution tecnique - DAMAS
One way to get rid of the __ is the Deconvolution Approach for the Mapping of Acoustic Sources (DAMAS)

In this approach we start considering the signal that a microphone placed at the center of the array would receive if a sound would be produced by the source $n$ of the searching grid.

$\begin{equation} P(\omega) = Q_n(\omega)e_{m:n}^{-1}(x, \omega) \end{equation}$

The power of the signal would be written as

$\begin{equation}
P_{m:n}(\omega)P_{m:n}^*(\omega) =
(Q_n(\omega)e_{m:n}^{-1}(x, \omega))^*(Q_n(\omega)e_{m:n}^{-1}(x, \omega)) = 
Q_n(\omega) Q_n^*(\omega) (e_{m:n}^{-1}(x, \omega))^* (e_{m:n}^{-1}(x, \omega))
\end{equation}$

The modeled Cross Spectral Martix of the signal that the microphone would receive is

$\begin{equation}
CSM^{mod}_n(\omega) = X_n
\begin{bmatrix} 
(e_1^{-1})^*(e_1^{-1})           & \dots         & (e_1^{-1})^*(e_M^{-1})         \\ 
\vdots                           & \ddots        & \vdots                         \\
(e_M^{-1})^*(e_1^{-1})           & \dots         & (e_M^{-1})^*(e_M^{-1})         \\
\end{bmatrix}_n
\end{equation}$

Where $X_n$ is the power-spectral density contribution of a source located at the grid position $n$.

Considering that the the sources placed on the search grid are independent, the modeled CSM can be computed summing all the contributions of the single sources.
$\begin{equation}
CSM^{mod}(\omega) = \sum_{n=1}^N CSM^{mod}_n(\omega)
\end{equation}$

The modeled power output of the beamformer can now be computed as before:

$\begin{equation}
L^{mod} = \frac{\hat{e}^T CSM^{mod}(\omega) \hat{e}}{M^2}
\end{equation}$

